In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [2]:
df = pd.read_csv('../data/myPersonality/mypersonality_final.csv', encoding='latin1')
print("The size of data is {0}".format(df.shape[0]))
labels = ['cEXT','cNEU','cAGR','cCON','cOPN']

The size of data is 9917


In [3]:
df2 = pd.read_csv('../data/Essays/essays2007.csv', encoding='latin1')
print("The size of data is {0}".format(df2.shape[0]))

The size of data is 2468


In [4]:
df4 = df[['STATUS',labels[0]]]
df4 = df4.rename(columns={'STATUS': "text", labels[0]: "label"})
df4.label = pd.Categorical(pd.factorize(df4.label)[0])
df4.label = df4.label.apply(int)

In [5]:
df3 = df2[['text',labels[0]]]
df3 = df3.rename(columns={labels[0]: "label"})

In [6]:
data = df4.append(df3, ignore_index=True)

In [7]:
data.shape

(12385, 2)

In [8]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(data['text'], data['label'], test_size=0.1)

In [9]:

train_df = pd.DataFrame({
    'text': train_inputs,
    'label':train_labels
})
print(Counter(train_df['label'].values))

eval_df = pd.DataFrame({
    'text': validation_inputs,
    'label': validation_labels
})
print(eval_df.head())

Counter({0: 6207, 1: 4939})
                                                   text  label
3422                               monday night malaise      1
6995  in times that are tough you really learn somet...      0
9192                                        Where am I?      0
5124  wishes *PROPNAME* wasn't stuck in Chicago...ho...      0
2764                                  and so it goes...      1


In [10]:
print('size of training set: %s' % (len(train_inputs)))
print('size of validation set: %s' % (len(validation_inputs)))

size of training set: 11146
size of validation set: 1239


In [11]:
import ktrain
from ktrain import text
MODEL_NAME = 'roberta-base'
t = text.Transformer(MODEL_NAME, maxlen=500, classes=[False, True])

using Keras version: 2.2.4-tf


In [12]:
trn = t.preprocess_train(train_inputs, train_labels)
val = t.preprocess_test(validation_inputs, validation_labels)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

preprocessing train...
language: en


preprocessing test...
language: en


In [13]:
learner.lr_find(show_plot=True, max_epochs=2)

simulating training for different learning rates... this may take a few moments...
Train for 1857 steps
Epoch 1/2
   0/1857 [..............................] - ETA: 0s

KeyError: 'loss'

In [9]:
learner.fit_onecycle(8e-5, 4)



begin training using onecycle policy with max lr of 8e-05...
Train for 1488 steps, validate for 166 steps
Epoch 1/4
   1/1488 [..............................] - ETA: 4:19:25

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[6,12,500,500] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node tf_roberta_for_sequence_classification/roberta/encoder/layer_._10/attention/self/MatMul (defined at /home/thomas/.local/lib/python3.7/site-packages/transformers/modeling_tf_bert.py:233) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Reshape_822/_536]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[6,12,500,500] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node tf_roberta_for_sequence_classification/roberta/encoder/layer_._10/attention/self/MatM